In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt

# datetime

In [11]:
data = pd.read_excel('supply_chain_network.xlsx')
data['start']=pd.to_datetime(data['START_']) #convert datadate to date fmt
data = data.replace('4000/01/01','2015/12/31')
data['end'] = pd.to_datetime(data['END_'])
# data.info()

# competitor

In [12]:
data_competitor = data.loc[data['REL_TYPE'] == 'COMPETITOR']
data_competitor = data_competitor[['ID','start','end','TICKER','TICKER.1']]
# data_competitor.info()

In [13]:
data_competitor.columns = ['ID','start', 'end','source_ticker','target_ticker']
data_competitor = data_competitor.reset_index()
data_competitor = data_competitor.drop(['index'], axis=1)
# data_competitor.head()

# competitor 2015

In [16]:
df_2015 = data_competitor.loc[(data_competitor['end'] > '2014-12-31') & (data_competitor['start'] < '2016-01-01' )]
df_2015['date'] = '2015-12-31'
df_2014 = data_competitor.loc[(data_competitor['end'] > '2013-12-31') & (data_competitor['start'] < '2015-01-01' )]
df_2014['date'] = '2014-12-31'
# # data_competitor_2013 = data_competitor.loc[(data_competitor['end'] > '2012-12-31')]
# # data_competitor_2013['date'] = '2013-12-31'

/Users/xingkong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/xingkong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [17]:
df_2014 = df_2014.dropna()
df_2015 = df_2015.dropna()
df_2014.to_pickle('data_competitor_2014.pkl')
df_2015.to_pickle('data_competitor_2015.pkl')

In [18]:
df_2014.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 983 entries, 0 to 1656
Data columns (total 6 columns):
ID               983 non-null object
start            983 non-null datetime64[ns]
end              983 non-null datetime64[ns]
source_ticker    983 non-null object
target_ticker    983 non-null object
date             983 non-null object
dtypes: datetime64[ns](2), object(4)
memory usage: 53.8+ KB


In [20]:
df_2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 886 entries, 4 to 1656
Data columns (total 6 columns):
ID               886 non-null object
start            886 non-null datetime64[ns]
end              886 non-null datetime64[ns]
source_ticker    886 non-null object
target_ticker    886 non-null object
date             886 non-null object
dtypes: datetime64[ns](2), object(4)
memory usage: 48.5+ KB


In [15]:
G = nx.from_pandas_edgelist(df, 'source_ticker', 'target_ticker', create_using = nx.Graph())
# G2014 = nx.from_pandas_edgelist(data_competitor_2014, 'source_ticker', 'target_ticker', edge_attr = 'date', create_using = nx.Graph())
# G2013 = nx.from_pandas_edgelist(data_competitor_2013, 'source_ticker', 'target_ticker', edge_attr = 'date', create_using = nx.Graph())

In [12]:
# degree, closeness, betweenness, pagerank
# degree = pd.Series(G.degree())
# degree = pd.DataFrame(degree, columns=['firms'])
# degree[['firms','degree']] = degree['firms'].apply(pd.Series)
# degree.set_index('firms', inplace=True)

# # closeness
# closeness = pd.Series(nx.closeness_centrality(G))
# closeness = pd.DataFrame(closeness, columns = ['closeness'])

# # betweenness
# betweenness = pd.Series(nx.betweenness_centrality(G, normalized=True))
# betweenness = pd.DataFrame(betweenness, columns=['betweenness'])

# #pagerank
# pagerank = pd.Series(nx.pagerank(G))
# pagerank = pd.DataFrame(pagerank, columns=['pagerank'])

In [13]:
# merge 2015
# data_merge_2015 = pd.concat([degree,closeness, betweenness, pagerank],axis =1)
# data_merge_2015['date'] = '2015-12-31'

#merge2014
# data_merge_2014 = pd.concat([degree,closeness, betweenness, pagerank],axis =1)
# data_merge_2014['date'] = '2014-12-31'

#merge2013
# data_merge_2013 = pd.concat([degree,closeness, betweenness, pagerank],axis =1)
# data_merge_2013['date'] = '2013-12-31'

In [14]:
import seaborn as sns; sns.set(color_codes=True)
# ax_closeness = sns.kdeplot(closeness['closeness'], shade = True)

In [15]:
# data3 = data_competitor_2013.copy()
# data3 = data3.dropna()

# G2013 = nx.from_pandas_edgelist(data3, 'source_ticker', 'target_ticker',
#                             create_using = nx.Graph())

# plt.figure(figsize=(12,12))
# layout = nx.kamada_kawai_layout(G2015)
# #layout = nx.spring_layout(G)
# nx.draw(G2015, layout, edge_color='#AAAAAA', with_labels = True)

In [16]:
import community
import collections

In [17]:
partition = community.best_partition(G)
values = [partition.get(node) for node in G.nodes()]
counter=collections.Counter(values)
print(counter)

Counter({1: 146, 0: 145, 5: 105, 2: 57, 12: 34, 7: 25, 25: 20, 14: 17, 8: 15, 16: 15, 11: 14, 24: 14, 9: 13, 4: 11, 22: 10, 3: 9, 10: 7, 17: 7, 6: 6, 13: 5, 26: 5, 19: 4, 23: 4, 18: 3, 20: 3, 15: 2, 21: 2})


In [18]:
# plt.figure(figsize=(20,20))
# sp = nx.spring_layout(G)
# nx.draw_networkx(G, pos=sp, with_labels=True, node_size=500, node_color=values)
# # plt.axes('off')
# plt.show()

In [47]:
color = nx.coloring.greedy_color(G, strategy='largest_first')

In [40]:
color.head()

,colors
IBM,0
MSFT,1
AAPL,0
DELL,1
INSY,0


In [48]:
nx.set_node_attributes(G, color,'community')

In [49]:
G.nodes['IBM']['community']

0

In [51]:
G_p = nx.complement(G)

In [38]:
preds = nx.resource_allocation_index(G)

In [20]:
preds

<generator object _apply_prediction.<locals>.<genexpr> at 0x1a248bf048>

In [52]:
potential = pd.DataFrame(pd.Series(G_p.edges()), columns = ['potential edges'])

In [53]:
potential = potential.set_index('potential edges')

In [54]:
potential['preferential_attachment'] = [i[2] for i in nx.preferential_attachment(G, potential.index)]

In [55]:
potential['resource_allocation_index'] = [i[2] for i in nx.resource_allocation_index(G, potential.index)]
potential['jaccard_coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, potential.index)]
potential['adamic_adar_index'] = [i[2] for i in nx.adamic_adar_index(G, potential.index)]
potential['cn_soundarajan_hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, potential.index)]
potential['ra_index_soundarajan_hopcroft'] = [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, potential.index)]
potential['within_inter_cluster'] = [i[2] for i in nx.within_inter_cluster(G, potential.index)]

In [56]:
potential.head()

,preferential_attachment,resource_allocation_index,jaccard_coefficient,adamic_adar_index,cn_soundarajan_hopcroft,ra_index_soundarajan_hopcroft,within_inter_cluster
potential edges,,,,,,,
"(TIT-IT, TTEC)",1,0.000000,0.000000,0.000000,0,0,0.0
"(TIT-IT, IBM)",211,0.004831,0.004739,0.187522,1,0,0.0
"(TIT-IT, CRAY)",2,0.000000,0.000000,0.000000,0,0,0.0
"(TIT-IT, DELL)",59,0.000000,0.000000,0.000000,0,0,0.0
"(TIT-IT, THOR)",1,0.000000,0.000000,0.000000,0,0,0.0


In [57]:
potential.describe().T

,count,mean,std,min,25%,50%,75%,max
preferential_attachment,156914.0,4.783181,72.140275,1.0,1.0,2.0,2.000000,21100.000000
resource_allocation_index,156914.0,0.002367,0.022987,0.0,0.0,0.0,0.004739,5.504831
jaccard_coefficient,156914.0,0.174366,0.309672,0.0,0.0,0.0,0.333333,1.000000
adamic_adar_index,156914.0,0.063255,0.120935,0.0,0.0,0.0,0.186851,15.747822
cn_soundarajan_hopcroft,156914.0,0.322355,0.528144,0.0,0.0,0.0,1.000000,17.000000
ra_index_soundarajan_hopcroft,156914.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
within_inter_cluster,156914.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
